Exercício 01 - Bruno Gazoni 7585037

### Primeiro, vamos importar as bibliotecas necessárias.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(500, 500, "Exercício Prático #01", None, None)
glfw.make_context_current(window)

### Capturando eventos de teclado e mouse

In [3]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
glfw.set_mouse_button_callback(window,mouse_event)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4.

In [4]:
vertex_code = """
        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [12]:
import math

#############################################################################################################
#1: MACACO HIDRÁULICO - BOMBA + CABO + BALÃO + TRIÂNGULO

vcirculo = 64 # define a "qualidade" do circulo
vretangulos = 8
vtriangulo = 3
vlinhas = 4

vtotal = vcirculo + vtriangulo + vretangulos + vlinhas

vertices = np.zeros(vtotal, [("position", np.float32, 2)])

#círculo I:
pi = 3.14
counter = 0
radius = 0.3
offset_x = +0.0
offset_y = -0.6

angle = 0.0
for counter in range(vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter] = [x+offset_x,y+offset_y]
    
print(f"vértices do círculo I: 0 a {counter}")
formercounter = counter
    
#triângulo II:
vertices[counter+1] = [ -0.15,-0.5]
vertices[counter+2] = [+0.0, -0.2]
vertices[counter+3] = [0.15, -0.5]
counter += 3

print(f"vértices do triângulo II: {formercounter} a {counter}")
formercounter = counter

#linha III
vertices[counter+1] = [+0.0, -0.6]
vertices[counter+2] = [-0.6, -0.6]
counter += 2

print(f"vértices da linha III: {formercounter} a {counter}")
formercounter = counter

#retângulo IV
vertices[counter+1] = [-0.9, -0.3] #G
vertices[counter+3] = [-0.6,-0.3] #J
vertices[counter+2] = [-0.9, -0.9] #H
vertices[counter+4] = [-0.6,-0.9] #I
counter += 4

print(f"vértices do retângulo IV: {formercounter} a {counter}")
formercounter = counter

#retângulo V
vertices[counter+1] = [-0.8, +0.0] #C
vertices[counter+2] = [-0.7, +0.0] #D
vertices[counter+3] = [-0.8,-0.3] #E
vertices[counter+4] = [-0.7,-0.3] #F
counter += 4

print(f"vértices do retângulo V: {formercounter} a {counter}")
formercounter = counter

#linha VI
vertices[counter+1] = [-0.9, +0.0]
vertices[counter+2] = [-0.6, +0.0]
counter += 2

print(f"vértices da linha VI: {formercounter} a {counter}")
formercounter = counter
#############################################################################################################


vértices do círculo I: 0 a 63
vértices do triângulo II: 63 a 66
vértices da linha III: 66 a 68
vértices do retângulo IV: 68 a 72
vértices do retângulo V: 72 a 76
vértices da linha VI: 76 a 78


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [15]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [17]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [18]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

### Nesse momento, nós exibimos a janela!


In [19]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
while not glfw.window_should_close(window):

    glfw.poll_events() 

#############################################################################################################
#1: MACACO HIDRÁULICO - BOMBA + CABO + BALÃO + TRIÂNGULO

    # vértices do círculo I: 0 a 63
    # vértices do triângulo II: 63 a 66
    # vértices da linha III: 66 a 68
    # vértices do retângulo IV: 68 a 72
    # vértices do retângulo V: 72 a 76
    # vértices da linha VI: 76 a 78
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0.5, 0.0, 0.5, 0.0)
    
    #circulo I
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 0, 64)
    
    #triangulo II
    glUniform4f(loc_color, 0.3, 0.0, 1.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 64, 3)
    
    #linha III
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0)
    glDrawArrays(GL_LINES, 67, 2)
    
    #retângulo IV
    glUniform4f(loc_color, 0.2, 0.0, 0.0, 0.2)
    glDrawArrays(GL_TRIANGLE_STRIP, 69, 4)
    
    #retângulo V
    glUniform4f(loc_color, 0.2, 0.0, 0.0, 0.2)
    glDrawArrays(GL_TRIANGLE_STRIP, 73, 4)
    
    #linha VI
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0)
    glDrawArrays(GL_LINES, 77, 2)
    
#############################################################################################################
    
    
    glfw.swap_buffers(window)

glfw.terminate()